In [ ]:
# Install dependencies
# !pip3 uninstall numpy
!pip3 install --upgrade numpy==2.0.0
!pip3 install pandas
!pip3 install scikit-learn mlflow seaborn shap
!pip3 install bayesian-optimization



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python310.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python310.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python310.exe -m pip install --upgrade pip


In [77]:
# Import library

import pandas as pd
import numpy as np

import mlflow
from mlflow import MlflowClient

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

import seaborn as sns
import matplotlib.pyplot as plt
import shap
import os

In [78]:
# Read dataset
# df = pd.read_csv('data/data.csv')
# train_test_data, validate_data = train_test_split(df, test_size=0.2, random_state=42)
# train_test_data.to_csv('data/train_.csv', index=False, header=True)
# validate_data.to_csv('data/validate.csv', index=False, header=True)

In [79]:
# Check dataset
df = pd.read_csv('data/train_.csv')
df.head()
print("Dataset column")
print(df.columns)
print("Summary of dataset info")
print(df.info)
print("view dimensions of dataset")
df.shape

# for col in df.columns:
#   if df[col].dtype != 'object':  # Exclude non-numeric columns
#     min_val = df[col].min()
#     max_val = df[col].max()
#     print(f"Column: {col}")
#     print(f"Minimum: {min_val}")
#     print(f"Maximum: {max_val}")
#     print()

print(df.isnull().sum())


Dataset column
Index(['ID', 'flow_duration', 'Header_Length', 'Protocol type', 'Duration',
       'Rate', 'Srate', 'Drate', 'fin_flag_number', 'syn_flag_number',
       'rst_flag_number', 'psh_flag_number', 'ack_flag_number',
       'ece_flag_number', 'cwr_flag_number', 'ack_count', 'syn_count',
       'fin_count', 'urg_count', 'rst_count', 'HTTP', 'HTTPS', 'DNS', 'Telnet',
       'SMTP', 'SSH', 'IRC', 'TCP', 'UDP', 'DHCP', 'ARP', 'ICMP', 'IPv', 'LLC',
       'Tot sum', 'Min', 'Max', 'AVG', 'Std', 'Tot size', 'IAT', 'Number',
       'Magnitue', 'Radius', 'Covariance', 'Variance', 'Weight', 'Label'],
      dtype='object')
Summary of dataset info
<bound method DataFrame.info of               ID  flow_duration  Header_Length  Protocol type  Duration  \
0        2696539       4.789412         108.00           6.00     64.00   
1        1487915      89.817633      122260.90           8.70    113.70   
2        2033215    1496.467448      211355.40           8.20     88.70   
3         13142

In [80]:
# Fill all null data as -1
print(df.isnull().sum())
data_n_null = df.fillna(-1, inplace=False)
print(data_n_null.isnull().sum())

data_n_null.head()
print(data_n_null.duplicated().sum())
print(data_n_null['Label'].unique().tolist())


ID                      0
flow_duration           0
Header_Length      155801
Protocol type      155810
Duration           156043
Rate               156180
Srate              156075
Drate              156049
fin_flag_number         0
syn_flag_number         0
rst_flag_number    156030
psh_flag_number    156006
ack_flag_number         0
ece_flag_number    155889
cwr_flag_number    156119
ack_count          156078
syn_count          156278
fin_count               0
urg_count               0
rst_count               0
HTTP               155993
HTTPS              156399
DNS                     0
Telnet             156044
SMTP               155832
SSH                156261
IRC                     0
TCP                156010
UDP                     0
DHCP                    0
ARP                156189
ICMP               155988
IPv                     0
LLC                     0
Tot sum            155800
Min                156138
Max                155961
AVG                     0
Std         

In [81]:
# Set mlflow as tracking server
mlflow.set_tracking_uri("http://localhost:5000")

In [ ]:
# Decision Tree Classification: https://gist.github.com/pb111/af439e4affb1dd94879579cfd6793770
mlflow.set_experiment("decision_tree")

tags = {
    "dataset_frac": 0.5,
    "random_state": 42,
    "test_size" : 0.2,
    # "droped_column" : ['ID'],
    "droped_column" : ['ID','IRC','DHCP','IPv', 'LLC', 'DNS','Drate','ece_flag_number', 'Telnet', 'ARP','rst_flag_number','psh_flag_number','ack_flag_number', 'SSH','HTTPS'],
    "author": "Son Nguyen",
    "parameter" : {
        "criterion" : ['gini','entropy','log_loss'],
        "max_depth" : [int(x) for x in np.linspace(10, 110, num = 11)],
        "min_samples_split" : [int(x) for x in np.random.randint(2, 21, 10)],
        "min_samples_leaf" : [int(x) for x in np.random.randint(2, 21, 10)]
    }
}

# Prepare data
data = data_n_null.drop(columns=tags['droped_column'])
data_sample = data.sample(frac=tags['dataset_frac'])
X = data_sample.drop(columns=['Label'])
y = data_sample['Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = tags['test_size'], random_state = tags['random_state'])

# Pick model
tree = DecisionTreeClassifier(random_state=tags['random_state'])
grid_search = GridSearchCV(
    estimator=tree,
    param_grid=tags['parameter'],
    cv=5,  
    scoring='f1',  
    verbose=2,
    n_jobs=-1  # Use all available CPU cores
)


with mlflow.start_run():
    # Turning hyper parameter
    mlflow.sklearn.autolog()
    grid_search.fit(X_train, y_train)
    best_tree = grid_search.best_estimator_
    best_params = grid_search.best_params_

    y_pred = best_tree.predict(X_test)

    # Create some artifact
    feature_scores = pd.Series(best_tree.feature_importances_, index=X_train.columns).sort_values(ascending=False)
    plt.figure(figsize=(20, 20))
    sns.barplot(x=feature_scores, y=feature_scores.index)
    plt.xlabel('Feature Importance Score')
    plt.ylabel('Features')
    plt.title("Visualizing Important Features")
    feature_importance_plot = "feature_importance.png"
    plt.savefig(feature_importance_plot, bbox_inches='tight')

    report_dict = classification_report(y_test, y_pred, output_dict=True)
    report_df = pd.DataFrame(report_dict).transpose()
    report_df.reset_index(inplace=True)
    report_df.rename(columns={"index": "Attack Type"}, inplace=True)
    report_filename = "classification_report.csv"
    report_df.to_csv(report_filename, index=False)

    mlflow.log_artifact(feature_importance_plot)
    mlflow.log_artifact(report_filename)
    os.remove(feature_importance_plot)
    os.remove(report_filename)

    for key, value in tags.items():
        if key != "parameter":
            mlflow.set_tag(key, value)
    
    mlflow.set_tag("best_param", best_params)

Fitting 5 folds for each of 3300 candidates, totalling 16500 fits
🏃 View run vaunted-shark-722 at: http://localhost:5000/#/experiments/5/runs/734e8f937a9f44e482774e141ce9df23
🧪 View experiment at: http://localhost:5000/#/experiments/5


KeyboardInterrupt: 

In [ ]:
# Naive Bayes https://gist.github.com/pb111/9e3816d2584a85ef7bff8d70bed20b1b

In [ ]:
# # # Random forest follow this: https://gist.github.com/pb111/88545fa33780928694388779af23bf58
# # Turning hyper parameter follow this: https://www.geeksforgeeks.org/random-forest-hyperparameter-tuning-in-python/
mlflow.set_experiment("random-forest")

tags = {
    "dataset_frac": 0.2,
    "random_state": 42,
    "test_size" : 0.2,
    "droped_column" : ['ID'],
    # "droped_column" : ['ID','IRC','DHCP','LLC','IPv','DNS','ece_flag_number','Drate','SMTP','Telnet','cwr_flag_number','ARP','SSH'],
    "author": "Son Nguyen",
    "parameter" : {
        "n_estimators" : [int(x) for x in np.linspace(start = 50, stop = 1000, num = 10)],
        "max_depth" : [int(x) for x in np.linspace(10, 110, num = 11)],
        # "min_samples_split" : [int(x) for x in np.linspace(2, 20, num = 10)],
        # "min_samples_leaf" : [round(x,5) for x in np.linspace(0.0001, 0.1, 10)],
        "max_features" : ['auto', 'sqrt', None],
        "criterion" : ['gini','entropy'],
        "max_leaf_nodes" : [int(x) for x in np.linspace(10, 100, num = 13)],
        # "bootstrap" : [True, False],
        # "n_jobs" : [-1, None]
    }
}
tags['parameter']['max_depth'].append(None)

# Prepare data
data = data_n_null.drop(columns=tags['droped_column'])
data_sample = data.sample(frac=tags['dataset_frac'])
X = data_sample.drop(columns=['Label'])
y = data_sample['Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = tags['test_size'], random_state = tags['random_state'])

# Pick model
model = RandomForestClassifier(random_state = tags['random_state'])
grid_search = GridSearchCV(
    estimator=model,
    param_grid=tags['parameter'],
    cv=5,  
    scoring='f1',  
    verbose=2,
    n_jobs=-1  # Use all available CPU cores
)

with mlflow.start_run():
    # Turning hyper parameter
    mlflow.sklearn.autolog()
    grid_search.fit(X_train, y_train)
    best_params = grid_search.best_params_
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)

    # Create some artifact
    feature_scores = pd.Series(best_model.feature_importances_, index=X_train.columns).sort_values(ascending=False)
    plt.figure(figsize=(20, 20))
    sns.barplot(x=feature_scores, y=feature_scores.index)
    plt.xlabel('Feature Importance Score')
    plt.ylabel('Features')
    plt.title("Visualizing Important Features")
    feature_importance_plot = "feature_importance.png"
    plt.savefig(feature_importance_plot, bbox_inches='tight')

    report_dict = classification_report(y_test, y_pred, output_dict=True, pos_label=None)
    report_df = pd.DataFrame(report_dict).transpose()
    report_df.reset_index(inplace=True)
    report_df.rename(columns={"index": "Attack Type"}, inplace=True)
    report_filename = "classification_report.csv"
    report_df.to_csv(report_filename, index=False)

    mlflow.log_artifact(feature_importance_plot)
    mlflow.log_artifact(report_filename)
    os.remove(feature_importance_plot)
    os.remove(report_filename)

    for key, value in tags.items():
        if key != "parameter":
            mlflow.set_tag(key, value)

    mlflow.set_tag("best_param", best_params)

Fitting 5 folds for each of 9360 candidates, totalling 46800 fits
🏃 View run calm-bass-784 at: http://localhost:5000/#/experiments/4/runs/9c80471f75ae47fd8f928b3b1198d810
🧪 View experiment at: http://localhost:5000/#/experiments/4


KeyboardInterrupt: 